In [1]:
import os

In [2]:
%pwd

'c:\\Users\\ainao\\Downloads\\Projects\\Sentiment Analysis of Amazon Reviews\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\ainao\\Downloads\\Projects\\Sentiment Analysis of Amazon Reviews'

In [5]:
import tensorflow as tf
import tensorflow_hub as hub

c:\Users\ainao\anaconda3\envs\sentA\lib\site-packages\tensorflow_hub\__init__.py:61: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    data_path: Path
    all_params: dict

In [7]:
from sentimentanalyzer.utils.common import read_yaml, create_directories  # adjust import as needed
from sentimentanalyzer.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH

In [8]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/model_trainer/sentiment_model",
            data_path="artifacts/data_transformation",
            all_params=self.params
        )
        return eval_config

In [9]:
import tensorflow as tf
import numpy as np
import json
from pathlib import Path
from sentimentanalyzer.utils.common import save_json, load_fasttext_file

In [10]:
class Evaluation:
    def __init__(self, config: 'EvaluationConfig'):
        self.config = config
        self.model = None
        self.test_ds = None
        self.score = None
       
        test_path = Path(config.data_path) / "test_ft.txt"
        self.test_texts, self.test_labels = load_fasttext_file(test_path)
        self.test_texts = self.test_texts
        self.test_labels = self.test_labels

    def load_model(self) -> tf.keras.Model:
        """
        Load and return the Keras model from the configured path.
        """
        model_path: Path = self.config.path_of_model
        self.model = tf.keras.models.load_model(model_path, custom_objects={'KerasLayer': hub.KerasLayer})
        return self.model

    def evaluate(self) -> tuple:
         self.load_model()
         self.score = self.model.evaluate(
             x =np.array(self.test_texts),
             y =np.array(self.test_labels)
         )
         self.save_score()
         return self.score

    
    def save_score(self, output_path: Path = Path("scores.json")) -> None:
        """
        Save evaluation results to a JSON file at output_path.
        """
        if self.score is None:
            raise ValueError("No evaluation score to save. Run evaluate() first.")

        loss, accuracy = self.score
        scores = {"loss": float(loss), "accuracy": float(accuracy)}
        save_json(path=output_path, data=scores)


In [11]:
try:
    config = ConfigurationManager()
    val_config = config.get_evaluation_config()
    evaluation = Evaluation(val_config)
    evaluation.load_model()
    evaluation.evaluate()
    evaluation.save_score()

except Exception as e:
   raise e

[2025-06-18 06:30:36,742: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-18 06:30:36,750: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-18 06:30:36,756: INFO: common: created directory at: artifacts]
4/4 [==============================] - 0s 8ms/step - loss: 0.6929 - accuracy: 0.5300
[2025-06-18 06:30:38,179: INFO: common: json file saved at: scores.json]
[2025-06-18 06:30:38,181: INFO: common: json file saved at: scores.json]
